In [1]:
!pip install transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer, TFAutoModelForSequenceClassification
import pathlib
import torch
import numpy as np
!pip install evaluate
import evaluate
import gensim.downloader
import gensim.models.keyedvectors as word2vec
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.2 MB/s eta 0:00:0

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [3]:
train_text=[]
train_sentiment=[]
test_text=[]
test_sentiment=[]
for sentiment in ["pos","neg"]:
  for text_file in (pathlib.Path('/content/drive/MyDrive/Colab Notebooks/aclImdb/train')/sentiment).iterdir():
    train_text.append(text_file.read_text())
    if sentiment=='pos':
      train_sentiment.append(0)
    else:
      train_sentiment.append(1)
print(train_text[0])

Here's another film that doesn't really need much of a recommendation. It's a classic comedy, very funny and entertaining and which, of course, ultimately inspired a successful television series which many would say was even better (I enjoy both, personally). <br /><br />For some, it's hard to warm up to Jack Lemmon and Walter Matthau as Felix Unger and Oscar Madison when they were were weaned on the TV show starring Tony Randall and Jack Klugman (or perhaps vice versa). But what we've got there in both cases are four good actors who in real life seemed so much like their film counterparts that they managed to make these characterizations their own. It's Neil Simon's humorous material that's key, and where the laughs really originate from.<br /><br />For those who have somehow never heard of THE ODD COUPLE, it's the story of a neurotic and fussy neat-freak (Lemmon) who is thrown out of a 12-year marriage by his long-suffering wife and takes up residence in the Manhattan apartment of hi

In [4]:
for sentiment in ["pos","neg"]:
  for text_file in (pathlib.Path('/content/drive/MyDrive/Colab Notebooks/aclImdb/test')/sentiment).iterdir():
    test_text.append(text_file.read_text())
    if sentiment=='pos':
      test_sentiment.append(0)
    else:
      test_sentiment.append(1)
print(test_text[0])

Having seen the original when I was 13 (and, yes, I was stupid enough to watch it while babysitting!), I was excited to see this remake.<br /><br />Camilla Bell did a great job as Jill Johnson. And the fact that a teen horror flick could be made in the year 2006 without tremendous vulgarity and gore, made it even that much stronger of a film. I had a great time trying not to chew my fingernails off!<br /><br />This film won't win anyone an Oscar, but it is entertaining and worth the matinée price ticket I bought to see it. I think girls around the world should watch the original and the remake...and then determine to never babysit again. <br /><br />All I can say is, I'm glad I'm too old to babysit! There's just something about being in a dark creepy house with sleeping kids that makes this movie classic. No blood, no gore...just good psychological fun! WINNER!


In [5]:
train_tokenized = tokenizer(train_text, truncation=True, padding=True)
test_tokenized = tokenizer(test_text, truncation=True, padding=True)

In [6]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_tokenized, train_sentiment)
# print(len(train_dataset))

test_dataset = IMDbDataset(test_tokenized, test_sentiment)
# print(len(test_dataset))

In [7]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
model1 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model1.classifier = torch.nn.Linear(768, 2)
model1.num_labels = 2
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)
trainer_without_freezing = Trainer(model=model1,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [9]:
trainer_without_freezing.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.288800,0.278975,0.920382
2,0.178900,0.300879,0.925216
3,0.054400,0.392651,0.928651


TrainOutput(global_step=9375, training_loss=0.187798505859375, metrics={'train_runtime': 4888.1387, 'train_samples_per_second': 15.343, 'train_steps_per_second': 1.918, 'total_flos': 9935054899200000.0, 'train_loss': 0.187798505859375, 'epoch': 3.0})

In [ ]:
# trainer.save_model("'/content/drive/MyDrive/Colab Notebooks/aclImdb/")

In [ ]:
# # model1 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
# from transformers import AutoModelForSequenceClassification
# model1 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
# from torch.optim import AdamW
# optimizer = AdamW(model1.parameters(), lr=5e-5)
# from transformers import get_scheduler
# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataset)
# lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model1.classifier=torch.nn.Linear(768,2)
# model1.num_labels = 2
# model1.to(device)
# from tqdm.auto import tqdm
# progress_bar = tqdm(range(num_training_steps))
# model1.train()
# for epoch in range(num_epochs):
#     for batch in train_dataset:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model1(**batch)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

  0%|          | 0/75000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [10]:
model2 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model2.classifier = torch.nn.Linear(768, 2)
model2.num_labels = 2
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)
trainer_with_freezing = Trainer(model=model2,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics,)
for param in model2.base_model.parameters():
    param.requires_grad = False
trainer_with_freezing.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

Epoch,Training Loss,Validation Loss,Accuracy
1,0.370400,0.342992,0.850551
2,0.372200,0.333339,0.857462
3,0.350100,0.332756,0.859020


TrainOutput(global_step=9375, training_loss=0.3767813395182292, metrics={'train_runtime': 2519.4881, 'train_samples_per_second': 29.768, 'train_steps_per_second': 3.721, 'total_flos': 9935054899200000.0, 'train_loss': 0.3767813395182292, 'epoch': 3.0})

In [11]:
word2vecmodel=gensim.downloader.load('glove-twitter-200')

[=================================================-] 99.8% 757.3/758.5MB downloaded


In [12]:
tokenize = Tokenizer(num_words=88583)
tokenize.fit_on_texts(train_text)
words_index=tokenize.word_index
train_x = tokenize.texts_to_sequences(train_text)
maxlen=200
# maxlen=max(len(text.split()) for text in train_text)
train_x=tf.keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
tokenize.fit_on_texts(test_text)
test_x = tokenize.texts_to_sequences(test_text)
# maxlen=max(len(text.split()) for text in test_text)
test_x=tf.keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
embedding_matrix = np.zeros((len(words_index) + 1, maxlen))
for word, i in words_index.items():
  if word in word2vecmodel:
    embedding_vector = word2vecmodel[word]
    embedding_matrix[i] = embedding_vector

In [13]:
embedding_layer = Embedding(input_dim=len(words_index) + 1,output_dim=maxlen,weights=[embedding_matrix],input_length=maxlen,trainable=False)
model3 = Sequential()
model3.add(embedding_layer)
model3.add(LSTM(units=100))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          17716600  
                                                                 
 lstm (LSTM)                 (None, 100)               120400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 17,837,101
Trainable params: 120,501
Non-trainable params: 17,716,600
_________________________________________________________________
None


In [14]:
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model3.fit(train_x,np.array(train_sentiment), epochs=3,validation_data=(test_x, np.array(test_sentiment)))

Epoch 1/3
782/782 [==============================] - 18s 16ms/step - loss: 0.5095 - accuracy: 0.7509 - val_loss: 0.8567 - val_accuracy: 0.5813
Epoch 2/3
782/782 [==============================] - 13s 17ms/step - loss: 0.3577 - accuracy: 0.8466 - val_loss: 0.8760 - val_accuracy: 0.5778
Epoch 3/3
782/782 [==============================] - 12s 15ms/step - loss: 0.3028 - accuracy: 0.8731 - val_loss: 0.8876 - val_accuracy: 0.5640


In [20]:
print("Fine-tuned model with all layers",)
print(trainer_without_freezing.evaluate(eval_dataset=test_dataset))
print("---------------------------")
print("Fine-tuned model after freezing the layers",)
print(trainer_with_freezing.evaluate(eval_dataset=test_dataset))
print("---------------------------")

Fine-tuned model with all layers


{'eval_loss': 0.39265093207359314, 'eval_accuracy': 0.928651326302333, 'eval_runtime': 407.0313, 'eval_samples_per_second': 61.499, 'eval_steps_per_second': 7.687, 'epoch': 3.0}
---------------------------
Fine-tuned model after freezing the layers


{'eval_loss': 0.33275556564331055, 'eval_accuracy': 0.8590204538191115, 'eval_runtime': 406.0888, 'eval_samples_per_second': 61.642, 'eval_steps_per_second': 7.705, 'epoch': 3.0}
---------------------------


In [22]:
trainer_without_freezing.save_model("Finetuning_1")
trainer_with_freezing.save_model("Finetuning_2")

In [17]:
loss, accuracy = model3.evaluate(test_x,np.array(test_sentiment))

783/783 [==============================] - 4s 5ms/step - loss: 0.8876 - accuracy: 0.5640
